## Install and imports

Explicación de los modulos usados y por que

### Imports

In [15]:
import pandas as pd
import numpy as np
import warnings
import re
import seaborn as sn
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#### Settings

In [ ]:
# Seaborn style
sn.set_theme()


# Color palette for Plotly
palette = ["#FC645F", "#A8E4A0", "#7088FF", "#FEB1AF"]


# Silencing warnings
warnings.filterwarnings('ignore')


# Setting max columns to six
pd.options.display.max_columns = 6


# Setting max rows to ten
pd.options.display.max_rows = 10

### Funciones



<b>viz_categorical</b> <br>
Esta función dinamicamente encuentra las variables catégoricas (con un value count de 25 o menos) de un dataframe y plotea los distintos valores en la serie. La utilizaremos para encontrar las variables categóricas sino tambien para marcar aquellas que son constantes, es decir, cuentan con un solo valor.

In [ ]:
def viz_categorical(dataframe):
    column_list = [column for column in dataframe.columns if len(dataframe[column].value_counts().index)>0 and len(dataframe[column].value_counts().index)<=40]
    context_dataframe = dataframe[column_list]
    
    
    # Calculating row index for plot based on x * 3 design
    if context_dataframe.shape[1] % 3 != 0:
        rows = round(context_dataframe.shape[1] / 3) + 1
    else:
        rows = round(context_dataframe.shape[1] / 3)


    # Plot constants
    fig, axes = plt.subplots(rows,3, figsize=(15, 10))
    fig.suptitle('Categorical variables')
    
    col_iter = 0
    row_iter = 0


    # Plotting
    for column, (i, ax) in zip(context_dataframe.columns, enumerate(fig.axes)):
        col_loc = context_dataframe.columns.get_loc(column)
        sn.barplot(x=context_dataframe[column].value_counts().index, y=context_dataframe[column].value_counts().values, ax=ax)
        ax.title.set_text(column)
        if len(context_dataframe[column].value_counts().index) > 5:
            ax.set_xticks([])

In [ ]:


def dat_insights(dataframe,sheet_name=False,rtn=True,dat=False,date_parse=False):
  if type(dataframe) == str:
    dataframe = data_capture(dataframe,sheet_name,date_parse).capture()
  if dat is not False:
    dataframe = column_cleaner(dataframe,[column for column in dataframe.columns if column in dat])
  null_values = dataframe.isnull().sum().sort_values(ascending=False)
  nulls_sum = (dataframe.isnull().sum()).sum()
  fields_sum = (dataframe.shape[0] * dataframe.shape[1]) - nulls_sum
  nulls_percentile = max(null_values)
  fills_percentile = dataframe.shape[0] - nulls_percentile
  if null_values.shape[0] > 15:
    null_values = null_values[null_values != 0]
  else:
    pass
  fig = make_subplots(rows=2,
                      cols=2,
                      subplot_titles=["<b>CORRECT ENTRIES</b>",
                                      "<b>CORRECT CELLS",
                                      "<b>NULLS/COLUMN</b>"],
                      specs=[[{'type':'domain'},{'type':'domain'}],
                             [{'type':'xy',"colspan":2},{}]])
  fig.add_trace(go.Pie(labels=["Null entries","Valid entries"],
                       values=[nulls_percentile,fills_percentile],
                       hole=0.4,
                       marker_colors=palette,
                       name="Correct entries",
                       title=dataframe.shape[0],legendgroup="apie"),1,1)
  fig.add_trace(go.Pie(labels=["Null cells",
                               "Valid cells"],
                       values=[nulls_sum,
                               fields_sum],
                       hole=0.4,
                       marker_colors=palette,
                       name="Correct cells",
                       title=dataframe.shape[0]*dataframe.shape[1],
                       legendgroup="pie"),1,2)
  fig.add_trace(go.Bar(x=null_values,
                       y=null_values.index,
                       name="Nulls/Column",
                       orientation="h",
                       marker_color="#FC645F",
                       showlegend=False),2,1)
  fig.update_traces(textposition='inside')
  fig.update_layout(height=600,
                    width=800,
                    title=f"<b>{sheet_name.upper()} INSIGHTS</b>",
                    yaxis_title="Columns",
                    xaxis_title="Nulls",
                    font_size=14)
  if rtn == True:
    fig.show(config={'modeBarButtonsToAdd':['drawline',
                                            'drawopenpath',
                                            'drawclosedpath',
                                            'drawcircle',
                                            'drawrect',
                                            'eraseshape']})
  else:
    return fig


def table_dtype(dataframe,rtn=True):
  # Función que crea reporte de tipo de variables y datos
  # dataframe = objet.dataframe
  # arg = True: use fig.show() otherwise it returns fig for object
  df_dtypes = dataframe.convert_dtypes()
  table_columns = [column for column in df_dtypes.dtypes.index.values]
  table_values = [str(value) for value in df_dtypes.dtypes]
  values = pd.Series(table_values)
  fig = make_subplots(rows=2,
                      cols=2,
                      specs=[[{"type":"table"},{"type":"domain"}],
                             [{"type":"table"},{"type":"domain"}]],
                      subplot_titles=["",
                                      "<b>TYPE",
                                      "",
                                      "<b>CLASS"])
  fig.add_trace(go.Pie(labels=values.value_counts().index,
                       values=values.value_counts(),
                       marker_colors=palette,
                       hole=0.4,
                       title=values.shape[0],
                       legendgroup="pie"),1,2)
  fig.update_traces(textposition='inside')
  fig.add_trace(go.Table(header=dict(values=['Column',
                                             'Data type'],
                                     fill_color="#A8E4A0",
                                     line_color='darkslategray'),
                         cells=dict(values=[table_columns,
                                            table_values],
                                    line_color='darkslategray')),1,1)
  fig.update_layout(height=500,width=1000,
                    title="<b>DATA TYPE ANALYSIS",
                    font_size=12)

  type_var = []
  for value in range(0,len(df_dtypes.dtypes)):
    if "Int" in str(df_dtypes.dtypes[value]):
      type_var.append("Numerical")
    else:
      if len(df_dtypes[df_dtypes.dtypes.index[value]].value_counts().index) <= 2:
        type_var.append("Numerical/Boolean")
      else:
        type_var.append("Categorical")

  pie_values = [values for values in [type_var.count("Categorical"), type_var.count("Numerical"), type_var.count("Numerical/Boolean")] if values !=0]
  pie_labels = str(set(type_var)).replace("{","").replace("}","").replace("'","").split(",")
  fig.add_trace(go.Table(header=dict(values=["Columns",
                                             "Classification"],
                                     fill_color="#A8E4A0",
                                     line_color='darkslategray'),
                         cells=dict(values=[df_dtypes.dtypes.index,type_var])),2,1)
  fig.add_trace(go.Pie(labels=pie_labels,
                       values=pie_values,
                       marker_colors=palette,
                       hole=0.4,
                       legendgroup="pie2"),2,2)
  fig.update_layout(height=800,width=1000,title="<b>DATA TYPE ANALYSIS",font_size=14)
  if rtn == True:
    fig.show()
  else:
    return fig


def correlation_matrix(dataframe, name, rtn=True):
  dataframe = dataframe.convert_dtypes()
  fig=make_subplots(cols=1,
                    rows=1)
  z,x = dataframe.corr(method="pearson"),dataframe.corr(method="pearson").columns
  fig.add_trace(go.Heatmap(z = z,
                           x = x,
                           y = x,
                           colorscale="Burg"))
  fig.update_layout(height=800,width=800,title=f"<b>{name} FEATURE CORRELATION MATRIX")
  if rtn == True:
    fig.show('png', engine='orca')
        # fig.show(config={'modeBarButtonsToAdd':['drawline',
        #                                     'drawopenpath',
        #                                     'drawclosedpath',
        #                                     'drawcircle',
        #                                     'drawrect',
        #                                     'eraseshape']})
  else:
    return fig

## Analisis de datos exploratorio

### Primeros pasos

Primero exportamos nuestro dataset como objecto panda.dataframe para realizar las primeras observaciones. Con la utilización del método append, head y tail, generamos una preview del dataframe basada en los diez primeros y últimos registros.

In [16]:
file = 'dataset.csv'
df = pd.read_csv(file)

df_preview = pd.concat([df.head(5), df.tail(5)])
df_preview

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...
121215,121215,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,...,93.0,7699.115044,9354.838710,NaN,NaN,10000.0,http://www.properati.com.ar/1cja2_venta_depart...,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,https://thumbs4.properati.com/1/bjms0KnaAnlNoQ...
121216,121216,sell,house,Beccar,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436080.0,NaN,NaN,...,360.0,1383.333333,1383.333333,NaN,NaN,NaN,http://www.properati.com.ar/1cja6_venta_casa_b...,Excelente e impecable casa en Venta en Las Lom...,Ruca Inmuebles | Venta | Lomas de San Isidro |...,https://thumbs4.properati.com/2/PCc3WuQDjpNZc4...
121217,121217,sell,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,3433775.0,"-34.5706388726,-58.4755963355",-34.570639,...,39.0,2858.695652,3371.794872,NaN,NaN,NaN,http://www.properati.com.ar/1cja7_venta_depart...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,https://thumbs4.properati.com/9/YAe_-2gRVykADP...
121218,121218,sell,apartment,Plaza Colón,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,NaN,NaN,NaN,...,48.0,1997.916667,1997.916667,NaN,NaN,NaN,http://www.properati.com.ar/1cja8_venta_depart...,"2 Amb al contrafrente, luminoso. El departame...",2 amb. C/ dep. de servicio al contrafrente| Re...,https://thumbs4.properati.com/8/Q12PTvU6BQJ0ib...
121219,121219,sell,apartment,Capital Federal,|Argentina|Capital Federal|,Argentina,Capital Federal,3433955.0,NaN,NaN,...,77.0,1675.324675,1675.324675,NaN,NaN,3500.0,http://www.properati.com.ar/1cjaa_venta_depart...,Departamento de 77 m2 apto profesional. Edific...,Departamento apto profesional en el Centro,https://thumbs4.properati.com/8/mNEsnQWqB7joBF...


Con un pedido de atributo columns generamos una lista de las columnas presentes y luego consultamos el atributo shape para observa la conformación del dataframe.

In [ ]:
df.columns

In [ ]:
df.shape

Al parecer contamos con dos columnas index debido a que el archivo csv incluía una columna de índice sin label. Esta columna es una primera candidata para ser eliminada, pero antes de proceder es necesario validar que efectivamente hay un match 1:1 entre ambas.
La validación mediante indexado booleano desmuestra que no parece haber ningún valor donde el índice del dataset no coincida con el índice genero al crear el dataframe, así que procedemos a su eliminación.

In [ ]:
df[df['Unnamed: 0'] != df.index]

In [ ]:
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])

Una observación rápida de las columnas properati_url e image_thumbnail parece indicar que se tratan de columnas relacionadas con la aplicación o método utilizado para realizar scrapping. Procedemos a eliminar ambas

In [ ]:
df.drop(columns=['properati_url', 'image_thumbnail'], inplace=True)

In [ ]:
df.info()

In [ ]:
df.describe()

### Analisis de variables catégoricas

Para reducir el lenght de nuestro dataset (en cuanto al número de columnas), una de las primeras acciones que podemos tomar es la que encontrar variables categóricas y entre estas aquellas que sean constantes, es decir que no muestren ninguna variabilidad dentro de la serie.
Para esto utilizamos la función <i>viz_categorical</i> que comentamos en el apartado de funciones.

Observamos que:
<ul>
  <li>Contamos con un número reducido de variables categóricas.</li>
  <li>De estas, <i>operation</i> y <i>country_name</i> son constantes y deberan ser eliminadas dado que no tienen ningún valor predictivo sobre nuestra variable target </li>
  <li>La variable <i>property_type</i> se encuentra desbalanceada para algunas de las categorías. Una posible solución seria simplificar la clasificación en <i>apartment</i> y <i>other</i> en referencia a otro tipo de edificación</li>
  <li>Por otro lado, <i>currency</i> cuenta con outliers representados por PEN y UYU. Además ARS representa una porción muy baja de los registros. Lo mejor es realizar una conversión montaria a USD de la variable precio para asegurar la consistencia y eliminar esta columna.</li>
</ul>

In [ ]:
viz_categorical(df)

<br>

Para eliminar las constantes usaremos un one-liner / lambda function que representa la función a continuacion:

```python
def constant_check(dataframe):
    if isinstance(dataframe, pd.DataFrame) is True:
        for column in dataframe.columns:
            if len(dataframe[column].value_counts().index) == 1:
                dataframe = dataframe.drop(columns=[column])

    return dataframe
```

In [ ]:
# Lambda function que realiza eliminación de series constantes
constant_check = lambda dataframe:  dataframe.drop(columns=[column for column in dataframe.columns if len(dataframe[column].value_counts().index) == 1]) if isinstance(dataframe, pd.DataFrame) is True else 'Not a dataframe'

# Realizamos la transformación
df = constant_check(df)
df.shape

# INPUT PRECIO

Vamos a buscar armar un nuevo Dataset sobre el cual trabajar, el mismo contara con todas las features de precios y las features categoricas 'desciption' y 'titles'.

# 

In [17]:
df_precio = df [['price','price_aprox_local_currency','price_aprox_usd','price_usd_per_m2','price_per_m2','description','title','surface_total_in_m2','surface_covered_in_m2']]

df_precio.head(5)

,price,price_aprox_local_currency,price_aprox_usd,price_usd_per_m2,price_per_m2,description,title,surface_total_in_m2,surface_covered_in_m2
0,62000.0,1093959.0,62000.0,1127.272727,1550.000000,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,55.0,40.0
1,150000.0,2646675.0,150000.0,NaN,NaN,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,NaN,NaN
2,72000.0,1270404.0,72000.0,1309.090909,1309.090909,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,55.0,55.0
3,95000.0,1676227.5,95000.0,NaN,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,NaN,NaN
4,64000.0,1129248.0,64000.0,1828.571429,1828.571429,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,35.0,35.0


In [ ]:
df_precio.isna().sum()

In [ ]:
df_precio['price_aprox_usd'].fillna(df_precio['price_usd_per_m2'].mul(df['surface_total_in_m2']))

In [ ]:
df_precio['price_aprox_usd'].isna().sum()

In [ ]:
Filas_precio_Nan = df_precio [['price','price_aprox_local_currency','price_aprox_usd','price_usd_per_m2','price_per_m2']].isna().all(axis=1).sum()

print(f'La cantidad de filas donde todas las features son Nans al mismo tiempo es de: {Filas_precio_Nan} de una cantidad de 20410 filas ')

# Regex PRECIO

In [18]:
PRECIO = df_precio['description'].str.contains('PRECIO').value_counts()

print(PRECIO)

precio = df_precio['description'].str.contains('precio').value_counts()

print(precio)


False    118751
True       2467
Name: description, dtype: int64
False    110686
True      10532
Name: description, dtype: int64


In [19]:
nan_values = df_precio[df_precio['price'].isna()]

In [20]:
nan_values.isna().sum()

price                         20410
price_aprox_local_currency    20410
price_aprox_usd               20410
price_usd_per_m2              20410
price_per_m2                  20410
description                       0
title                             0
surface_total_in_m2            7384
surface_covered_in_m2          6757
dtype: int64

In [21]:
PRECIO = nan_values['description'].str.contains('PRECIO').value_counts()

print(PRECIO)

precio = nan_values['description'].str.contains('precio').value_counts()

print(precio)

usd = nan_values['description'].str.contains('usd').value_counts()

print(usd)

False    19928
True       482
Name: description, dtype: int64
False    18584
True      1826
Name: description, dtype: int64
False    20394
True        16
Name: description, dtype: int64


In [ ]:
# # reemplazo U$S por vacío
# dolares_pattern = "U\$S\s*"
# dolares_regex = re.compile(dolares_pattern)

# # reemplazo el punto decimal por vacío
# decimal_pattern = "\."
# decimal_regex = re.compile(decimal_pattern)

# sin_dolar = data.PRECIOTEXT.apply(lambda x: x if x is np.NaN else dolares_regex.sub("", x))

# sin_dolar_sin_punto = sin_dolar.apply(lambda x: x if x is np.NaN else decimal_regex.sub("", x))

In [ ]:
# def re_precios (precios):
#     return re.findall('\$\d+(?:\.\d+)?', precios)
#     # return re.findall ('^\$?\-?([1-9]{1}[0-9]{0,2}(\,\d{3})*(\.\d{0,2})?|[1-9]{1}\d{0,}(\.\d{0,2})?|0(\.\d{0,2})?|(\.\d{1,2}))$|^\-?\$?([1-9]{1}\d{0,2}(\,\d{3})*(\.\d{0,2})?|[1-9]{1}\d{0,}(\.\d{0,2})?|0(\.\d{0,2})?|(\.\d{1,2}))$|^\(\$?([1-9]{1}\d{0,2}(\,\d{3})*(\.\d{0,2})?|[1-9]{1}\d{0,}(\.\d{0,2})?|0(\.\d{0,2})?|(\.\d{1,2}))\)$',precios)
#     # return re.findall ('($usd|$USD|U$D)\s?([0-9]+\.+[0-9])',precios)
#     # return re.findall((\$(([0-9]?)[a-zA-Z]+)([0-9]?)))
#     # return re.findall('($usd|$USD|U$D)\s?(\d{1,}(?:[.,]\d{3})*(?: [.,]\d{2}))|(\d{1,3}(?:[.,]\d{3})*(?:[.,]\d{2})?)\ s?(USD|EUR)', precios)

In [ ]:
# # ^U\$D\s*?([0-9]{1,3}.([0-9]{3}.)*[0-9]{3}|[0-9]+)(,[0-9][0-9])?$

# def re_precios (precios):
#     return re.findall('^U\$D\s*?([0-9]{1,3}.([0-9]{3}.)*[0-9]{3}|[0-9]+)(,[0-9][0-9])?$', precios)

In [ ]:
Sample15 = df_precio_regex.sample(15)

Sample15 = Sample15.convert_dtypes()

In [ ]:
re_precios(Sample15)

In [ ]:
df_precio_regex = df [['price','description','price_aprox_usd']]

In [ ]:
df_precio_regex.dtypes

In [ ]:
df_precio_regex['description'] = df_precio_regex['description'].astype('str') 

In [ ]:
# df_precio_regex['precio_regex'] = df_precio_regex['description'].apply(re_precios)

# df_precio_regex

In [22]:
pattern_depto = "U\$D\s*?([0-9]{1,3}.([0-9]{3}.)*[0-9]{3}|[0-9]+)(,[0-9][0-9])?$"
pattern_depto_regex = re.compile (pattern_depto,flags = re.IGNORECASE)
desc_string = nan_values['description'].astype(str)
tiene_balcon = desc_string.apply(lambda x: pattern_depto_regex.findall(x))

In [23]:
tiene_balcon.value_counts()

# df['state_name'].value_counts().index.tolist()

[]                 20384
[(248.000, , )]        3
[(345.000, , )]        2
[(340.000, , )]        2
[(260.000, , )]        2
[(245.000, , )]        2
[(240.000, , )]        2
[(297000, , )]         1
[(255.000, , )]        1
[(350.000, , )]        1
[(230.000, , )]        1
[(225.000, , )]        1
[(395.000, , )]        1
[(450.000, , )]        1
[(335.000, , )]        1
[(200.000, , )]        1
[(330.000, , )]        1
[(220.000, , )]        1
[(150000, , )]         1
[(90.000, , )]         1
Name: description, dtype: int64

In [24]:
x = tiene_balcon.value_counts()[tiene_balcon.value_counts() < 20384]

x.index[0][0][0]


'248.000'

In [25]:
tiene_balcon

5         []
20        []
41        []
62        []
67        []
          ..
121210    []
121211    []
121212    []
121213    []
121214    []
Name: description, Length: 20410, dtype: object

# WIP

In [ ]:
columns = ['property_type', 'rooms']


sn.set_theme()
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Multi variate analysis based on price')
index = 0
for column in columns:
    group_df = df[[column, 'price']]
    mean_df = group_df.groupby(by=[column]).agg({"price":['mean', 'count']})
    sn.barplot(x=mean_df.index.tolist(), y=mean_df['price']['mean'].tolist(), ax=axes[0,index])
    
    
    index += 1









In [ ]:
# df['rooms'] = df['rooms'].fillna(value=0).astype(int)

# df.rooms.value_counts().sort_values()



### Validación mediante la columna description

A través de la columna description validaremos varias de las otras columnas, puesto que estas parecen ser una descomposición de la primera. Por este motivo,  lo primero que haremos es descartar aquellos registros cuya descripción es nula.

In [ ]:
df.drop(index=df[df['description'].isnull() == True].index, inplace=True)
df[df['description'].isnull() == True].index

In [ ]:
x = df_test['description'].apply(lambda x: x.lower().find('venta') if str(x) != 'nan' else 'nan') #.apply(lambda x: False if x == '-1' or x == 'nan' else True)
df = df.drop(index=x[x==False].index.tolist())


In [ ]:
# que dropear:
# operation
# lat-long sacarla
# price_aprox_local_curerncy
# property_url
# image thumbnail
# rooms
# country_name

#armar:
# ambientes a partir de description



df['state_name'].value_counts()

In [ ]:
table_dtype(df)

In [ ]:
test = correlation_matrix(df, 'PROPERATY', rtn=False)

In [ ]:
import plotly.io as pio
#
pio.show(test, engine='kaleido')


In [ ]:
df_1 = df[['state_name', 'price']]
df_1

In [ ]:

#x = dat_insights(df, 'EDA', rtn=True)
#x

In [ ]:
# Probando

